In [1]:

import torch.nn as nn
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# Define the root directory of the dataset
root_dir = "data/"

# Define the batch size for the data loader
batch_size = 32

# Define the transforms for the dataset
train_transforms = transforms.Compose([
    
    transforms.ToTensor()
])

val_transforms = transforms.Compose([
    
   
    transforms.ToTensor()
])

# Create the train and validation datasets
train_dataset = ImageFolder(root_dir + "train", transform=train_transforms)
val_dataset = ImageFolder(root_dir + "validation", transform=val_transforms)

# Create the train and validation data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [2]:
import torch.nn.functional as F

class Autoencoder(nn.Module):
    def __init__(self, encoding_dim):
        super(Autoencoder, self).__init__()
        self.encoding_dim = encoding_dim

        # Encoder layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=2, padding=1)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=2, padding=1)

        # Decoder layers
        self.deconv1 = nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=4, stride=2, padding=1)
        self.deconv2 = nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=4, stride=2, padding=1)
        self.deconv3 = nn.ConvTranspose2d(in_channels=64, out_channels=32, kernel_size=4, stride=2, padding=1)
        self.deconv4 = nn.ConvTranspose2d(in_channels=32, out_channels=3, kernel_size=4, stride=2, padding=1)

    def encode(self, x):
        # Encoder
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        encoded = F.relu(self.conv4(x))
        return encoded

    def decode(self, x):
        # Decoder
        x = F.relu(self.deconv1(x))
        x = F.relu(self.deconv2(x))
        x = F.relu(self.deconv3(x))
        decoded = torch.sigmoid(self.deconv4(x))
        return decoded

    def forward(self, x):
        encoded = self.encode(x)
        decoded = self.decode(encoded)
        return decoded, encoded

In [3]:
autoencoder=Autoencoder(256*256*256)

In [4]:
criterion = nn.MSELoss()


In [6]:
import torch.optim as optim
optimizer = optim.SGD(autoencoder.parameters(), lr=0.01, momentum=0.9)


In [8]:

# Train the ImageClassifier
num_epochs = 10
for epoch in range(num_epochs):
    # Training
    running_loss = 0.0
    autoencoder.train() # set the model to training mode
    for i, data in enumerate(train_loader, 0):
        inputs, _ = data
        optimizer.zero_grad()
        outputs, _ = autoencoder(inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_loss = running_loss/len(train_loader)

    # Validation
    running_loss = 0.0
    autoencoder.eval() # set the model to evaluation mode
    with torch.no_grad():
        for i, data in enumerate(val_loader, 0):
            inputs, _ = data
            outputs, _ = autoencoder(inputs)
            loss = criterion(outputs, inputs)
            running_loss += loss.item()
        val_loss = running_loss/len(val_loader)
       
    print('Epoch [%d], Training Loss: %.3f, Validation Loss: %.3f' % (epoch+1, train_loss, val_loss))

Epoch [1], Training Loss: 0.088, Validation Loss: 0.085
Epoch [2], Training Loss: 0.085, Validation Loss: 0.085
Epoch [3], Training Loss: 0.085, Validation Loss: 0.085
Epoch [4], Training Loss: 0.085, Validation Loss: 0.085
Epoch [5], Training Loss: 0.085, Validation Loss: 0.085
Epoch [6], Training Loss: 0.085, Validation Loss: 0.085
Epoch [7], Training Loss: 0.085, Validation Loss: 0.085
Epoch [8], Training Loss: 0.085, Validation Loss: 0.085
Epoch [9], Training Loss: 0.085, Validation Loss: 0.085
Epoch [10], Training Loss: 0.085, Validation Loss: 0.085


In [9]:
# Assuming your model is named 'model'
torch.save(autoencoder.state_dict(), 'auto.pth')


In [10]:
class Classifier(nn.Module):
    def __init__(self, input_dim):
        super(Classifier, self).__init__()
        
        # Define the convolutional layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=2, padding=1)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=2, padding=1)
        
        # Define the fully-connected layers
        self.fc1 = nn.Linear(in_features=input_dim, out_features=512)
        self.fc2 = nn.Linear(in_features=512, out_features=10)

    def forward(self, x):
        # Pass the input through the convolutional layers
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        
        # Flatten the output of the last convolutional layer
        x = x.view(x.size(0), -1)
        
        # Pass the flattened output through the fully-connected layers
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        
        return x


In [11]:
# Create a classifier and load the weights of the encoder from the autoencoder
classifier = Classifier(16*16*256)
classifier.conv1.load_state_dict(autoencoder.conv1.state_dict())
classifier.conv2.load_state_dict(autoencoder.conv2.state_dict())
classifier.conv3.load_state_dict(autoencoder.conv3.state_dict())
classifier.conv4.load_state_dict(autoencoder.conv4.state_dict())


<All keys matched successfully>

In [12]:
# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)

for epoch in range(10):
    # Training
    train_loss = 0.0
    train_acc = 0.0
    classifier.train() # set the model to training mode
    for images, labels in train_loader:


        # Forward pass
        outputs = classifier(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Compute training accuracy
        _, predicted = torch.max(outputs.data, 1)
        train_acc += (predicted == labels).sum().item()

        train_loss += loss.item() * images.size(0)
    train_loss /= len(train_loader)
    train_acc /= len(train_loader)

    # Validation
    val_loss = 0.0
    val_acc = 0.0
    classifier.eval() # set the model to evaluation mode
    with torch.no_grad():
        for images, labels in val_loader:


            # Forward pass
            outputs = classifier(images)
            loss = criterion(outputs, labels)

            # Compute validation accuracy
            _, predicted = torch.max(outputs.data, 1)
            val_acc += (predicted == labels).sum().item()

            val_loss += loss.item() * images.size(0)
        val_loss /= len(val_loader.dataset)
        val_acc /= len(val_loader.dataset)

    # Print the loss and accuracy after every epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")


Epoch [1/10], Training Loss: 22.1162, Training Accuracy: 17.9973, Validation Loss: 0.6320, Validation Accuracy: 0.6407
Epoch [2/10], Training Loss: 19.2683, Training Accuracy: 21.5394, Validation Loss: 0.5681, Validation Accuracy: 0.7014
Epoch [3/10], Training Loss: 16.4586, Training Accuracy: 23.7423, Validation Loss: 0.4651, Validation Accuracy: 0.7724
Epoch [4/10], Training Loss: 12.7222, Training Accuracy: 26.1188, Validation Loss: 0.4033, Validation Accuracy: 0.8165
Epoch [5/10], Training Loss: 8.5697, Training Accuracy: 28.3378, Validation Loss: 0.4214, Validation Accuracy: 0.8199
Epoch [6/10], Training Loss: 5.3865, Training Accuracy: 29.8331, Validation Loss: 0.4355, Validation Accuracy: 0.8356
Epoch [7/10], Training Loss: 3.2609, Training Accuracy: 30.7557, Validation Loss: 0.5288, Validation Accuracy: 0.8426
Epoch [8/10], Training Loss: 2.4511, Training Accuracy: 31.1308, Validation Loss: 0.5414, Validation Accuracy: 0.8391
Epoch [9/10], Training Loss: 1.9902, Training Accura

In [13]:
torch.save(classifier.state_dict(), 'classifier.pth')

In [14]:
# Define the dataset
dataset = ImageFolder(root='data2', transform = transforms.Compose([
                transforms.ToTensor()
            ])
)

# Define the data loader
test_loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [15]:
# Test the model
classifier.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        # Move the data to the device


        # Forward pass
        outputs = classifier(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Accuracy: {100 * correct / total:.2f}%")


Accuracy: 49.55%


In [22]:
# Load the image and preprocess it
from PIL import Image
import numpy as np

image = Image.open('train/cat.90.jpg')
image = image.resize((256, 256)) # Resize to the same size used during training
image = np.array(image)/255.0 # Normalize the pixel values between 0 and 1
image = np.transpose(image, (2, 0, 1)) # Convert to the shape (C, H, W) expected by PyTorch
image_tensor = torch.tensor(image).float() # Convert to tensor and change the data type to float

# Make a prediction
output = classifier(image_tensor.unsqueeze(0)) # Add an extra dimension to represent the batch size
probabilities = torch.softmax(output, dim=1)
predicted_class = torch.argmax(probabilities, dim=1)

# Print the predicted class and the associated probability
print(f"Predicted class: {'dog' if predicted_class == 0 else 'cat'}")


Predicted class: dog
